# FDS Mini Project


**WARNING: Before making any git commit to this notebook please clear all output in this notebook**

## 1. Cleaning the data

### Invalid Columns: 
- delete unnamed column which was serving as index (index already exists - duplicated column)
- delete last column (contains only NaN values) - 'Unnamed 21'

### NaN values:
- check number of NaN values/location of NaN values
- leave NaN values that are required in order not to lose data (for example: a cancelled flight will always have NaN values for DEP_TIME, ARR_TIME, ARR_DEL15, DEP_DEL15 - as the flight did not happen)
- delete NaN values that would incommodate analysis and plotting later on (for example, flight timings that are simply missing without the flight having been cancelled)

### Times conversion (Note: 00:00 timings all represent cancelled flights)
- observation --> no flight leaves at 00:00, all *00:00 date/time values belong to flights that have been cancelled*
- converted DEP_TIME and ARR_TIME to 4-character string of the format: hhmm (error when attempting to convert to date/time) 
- added two extra columns: ARR_TIME_MINS and DEP_TIME_MINS representing the arrival and departure time in minutes for easier calculations

### Irrelevant columns (to this project) to be removed/ duplicated data:
- Remove both OP_CARRIER_AIRLINE_ID and OP_CARRIER
- Remove ORIGIN_AIRPORT_SEQ_ID
- Remove DEST_AIRPORT_SEQ_ID





In [63]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

#--------------------------------------- Load dataset ------------------------------------------#
flight_data_path = os.path.join(os.getcwd(), 'datasets', 'flight_jan_2019.csv.gz')
flight_data = pd.read_csv(flight_data_path, compression = 'gzip')

# Delete 'Unnamed 1' and 'Unnamed 21'
del flight_data['Unnamed: 0']
del flight_data['Unnamed: 21']
flight_data

#---------------------------------------- Check for 'NaN' values ------------------------------#

# for col in flight_data.columns: 
#    print(col, ' :',flight_data[col].isna().sum())
    
    # NA VALUES: TAIL_NUM  : 2543
    #            DEP_TIME  : 16352
    #            DEP_DEL15  : 16355
    #            ARR_TIME  : 17061
    #            ARR_DEL15  : 18022
    #            Unnamed: 21  : 583985

# Dealing with DEP_TIME and ARR_TIME Nan values
flight_data[np.isnan(flight_data.DEP_TIME)] # Observation: cancelled flights have Nan values for DEP_TIME, ARR_TIME, DEP_DEL15,ARR_DEL15  
# NaN values therefore make sense in this case, eliminating rows with NaN values with plotting can be done by filtering:
#                       flight_data[~np.isnan(flight_data['DEP_TIME'])]['DEP_TIME'].isna().sum()    

# Eliminate rows with NaN values in place for DEP/ARR_DELL15 AND ARR_TIME where the DEP_TIME is registered (timings simply missing)
indices_to_eliminate = list(flight_data[(~np.isnan(flight_data['DEP_TIME']))][np.isnan(flight_data['DEP_DEL15'])].index.values) + list(flight_data[(~np.isnan(flight_data['DEP_TIME']))][np.isnan(flight_data['ARR_TIME'])].index.values) + list(flight_data[(~np.isnan(flight_data['DEP_TIME']))][np.isnan(flight_data['ARR_DEL15'])].index.values)
flight_data = flight_data.drop(indices_to_eliminate)

#--------------------------------------Modifying data types----------------------------------#
flight_data.dtypes
# CANCELLED/DIVERTED to integer value
flight_data['CANCELLED'] = flight_data['CANCELLED'].astype(int)
flight_data['DIVERTED'] = flight_data['DIVERTED'].astype(int)
flight_data.dtypes
flight_data
# Modifying timings date/time format
#flight_data['DEP_TIME'] = pd.to_datetime(flight_data['DEP_TIME'], format='%H%M').dt.time

# OBSERVATION: flights with value 0.0 - keeping in mind that timings are currently floats - are all NaN values - so no flight leaves at 00:00 (those are simply cancelled values)
len(flight_data[(flight_data['DEP_TIME'] == 0.0)][flight_data['CANCELLED'] == 1]['DEP_TIME']) - flight_data[flight_data['DEP_TIME'] == 0.0]['DEP_TIME'].isna().sum()
len(flight_data[(flight_data['DEP_TIME'] == 0.0)][flight_data['CANCELLED'] == 1]['DEP_TIME']) - flight_data[flight_data['DEP_TIME'] == 0.0]['DEP_TIME'].isna().sum()

# Convert DEP_TIME and ARR_TIME to int and add new columns: DEP_TIME_MINS and ARR_TIME_MINS for easy calculations
def convert_minutes(x):
    minutes = int(x[2])*10 + int(x[3])
    hr_minutes = (int(x[0])*10 + int(x[1]))*60
    return minutes+hr_minutes

def fill_in(x):
    if (len(x) == 4):
        return x
    if (len(x) == 3):
        return '0' + x
    if (len(x) == 2):
        return '00' + x
    if (len(x) == 1):
        return '000' + x
    if (len(x) == 0):
        return '000' + x
    return '0000'
    
flight_data['DEP_TIME'] = flight_data['DEP_TIME'].fillna(0)
flight_data['DEP_TIME'] = flight_data['DEP_TIME'].astype(int)
flight_data['DEP_TIME'] = flight_data['DEP_TIME'].astype(str)
flight_data['DEP_TIME'] = flight_data['DEP_TIME'].apply(fill_in)
flight_data['DEP_TIME_MINS'] = flight_data['DEP_TIME'].apply(convert_minutes)
flight_data['ARR_TIME'] = flight_data['ARR_TIME'].fillna(0)
flight_data['ARR_TIME'] = flight_data['ARR_TIME'].astype(int)
flight_data['ARR_TIME'] = flight_data['ARR_TIME'].astype(str)
flight_data['ARR_TIME'] = flight_data['ARR_TIME'].apply(fill_in)
flight_data['ARR_TIME_MINS'] = flight_data['ARR_TIME'].apply(convert_minutes)

#-------------------------------ATTEMPT AT CONVERTING TO DATE/TIME-----------------#
def fill_in(x):
    if (len(x) == 4):
        return x
    if (len(x) == 3):
        return '0' + x
    if (len(x) == 2):
        return '00' + x
    if (len(x) == 1):
        return '000' + x
    if (len(x) == 0):
        return '000' + x
    return '0000'
    
#def convert_time(x):
#    return datetime.datetime.strptime(x,'%H%M' )
    
#flight_data['DEP_TIME'] = flight_data['DEP_TIME'].apply(fill_in)
#flight_data['ARR_TIME'] = flight_data['ARR_TIME'].apply(fill_in)
#flight_data['DEP_TIME'] = flight_data['DEP_TIME'].apply(convert_time)
#flight_data['DEP_TIME'] = flight_data['DEP_TIME'].apply(check)
#flight_data['DEP_TIME'] = pd.to_datetime(flight_data['DEP_TIME'], format=)


#------------------------------------Eliminating extra columns------------------------------#

flight_data['OP_UNIQUE_CARRIER'].nunique()  # 17
flight_data['OP_CARRIER_AIRLINE_ID'].nunique()  # 17
flight_data['OP_CARRIER'].nunique() # 17
# Remove both OP_CARRIER_AIRLINE_ID and OP_CARRIER
del flight_data['OP_CARRIER_AIRLINE_ID']
del flight_data['OP_CARRIER']

flight_data['TAIL_NUM'].nunique() # 5445
flight_data['ORIGIN_AIRPORT_ID'].nunique() # 346
flight_data['ORIGIN_AIRPORT_SEQ_ID'].nunique() # 346
# Remove ORIGIN_AIRPORT_SEQ_ID
del flight_data['ORIGIN_AIRPORT_SEQ_ID']

flight_data['DEST_AIRPORT_ID'].nunique() # 346
flight_data['DEST_AIRPORT_SEQ_ID'].nunique() # 346
# Remove DEST_AIRPORT_SEQ_ID
del flight_data['DEST_AIRPORT_SEQ_ID']

flight_data


<ipython-input-63-4d802f8bc679>:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indices_to_eliminate = list(flight_data[(~np.isnan(flight_data['DEP_TIME']))][np.isnan(flight_data['DEP_DEL15'])].index.values) + list(flight_data[(~np.isnan(flight_data['DEP_TIME']))][np.isnan(flight_data['ARR_TIME'])].index.values) + list(flight_data[(~np.isnan(flight_data['DEP_TIME']))][np.isnan(flight_data['ARR_DEL15'])].index.values)
<ipython-input-63-4d802f8bc679>:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(flight_data[(flight_data['DEP_TIME'] == 0.0)][flight_data['CANCELLED'] == 1]['DEP_TIME']) - flight_data[flight_data['DEP_TIME'] == 0.0]['DEP_TIME'].isna().sum()
<ipython-input-63-4d802f8bc679>:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(flight_data[(flight_data['DEP_TIME'] == 0.0)][flight_data['CANCELLED'] == 1]['DEP_TIME']) - flight_data[flight_data['DEP_TIME'] == 0.0]['DEP_TIME'].

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,DEP_TIME_MINS,ARR_TIME_MINS
0,1,2,9E,N8688C,3280,11953,GNV,10397,ATL,0601,0.0,0600-0659,0722,0.0,0,0,300.0,361,442
1,1,2,9E,N348PQ,3281,13487,MSP,11193,CVG,1359,0.0,1400-1459,1633,0.0,0,0,596.0,839,993
2,1,2,9E,N8896A,3282,11433,DTW,11193,CVG,1215,0.0,1200-1259,1329,0.0,0,0,229.0,735,809
3,1,2,9E,N8886A,3283,15249,TLH,10397,ATL,1521,0.0,1500-1559,1625,0.0,0,0,223.0,921,985
4,1,2,9E,N8974C,3284,10397,ATL,11778,FSM,1847,0.0,1900-1959,1940,0.0,0,0,579.0,1127,1180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583980,31,4,UA,N776UA,200,12016,GUM,12173,HNL,0749,0.0,0700-0759,1832,0.0,0,0,3801.0,469,1112
583981,31,4,UA,N36280,174,12016,GUM,14955,SPN,0717,0.0,0700-0759,0759,0.0,0,0,129.0,437,479
583982,31,4,UA,N36280,117,14955,SPN,12016,GUM,0857,0.0,0900-0959,0933,0.0,0,0,129.0,537,573
583983,31,4,UA,N39726,105,14955,SPN,12016,GUM,1820,0.0,1800-1859,1854,0.0,0,0,129.0,1100,1134
